In [2]:
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
from datetime import datetime, date, timedelta

from math import log, sqrt, pi, exp
from scipy.stats import norm
#from pandas import DataFrame
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yfin
yfin.pdr_override()

pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
plt.rcParams["figure.figsize"] = (10,6)
import warnings

from pykalman import KalmanFilter
import numpy as np

from darts.models import KalmanForecaster
from darts.utils.timeseries_generation import datetime_attribute_timeseries, constant_timeseries
from darts.timeseries import TimeSeries as ts


/Users/dsimbandumwe/anaconda3/envs/DATA_618/lib/python3.10/site-packages/statsforecast/core.py:25: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [3]:
start = datetime(2023, 1, 1)
today = datetime.today()

sp_list = ['IVV']

In [4]:
df = pdr.get_data_yahoo(sp_list, start=start, end=today)
df.tail(20)

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-10-05,426.28,427.23,423.04,426.36,426.36,4030400
2023-10-06,423.87,433.02,422.46,431.37,431.37,4153500
2023-10-09,429.45,434.79,428.92,434.21,434.21,2675500
2023-10-10,434.89,439.15,434.47,436.48,436.48,4430000
2023-10-11,437.63,438.50,435.10,438.33,438.33,3875600
2023-10-12,438.91,439.28,433.17,435.70,435.70,4232300
2023-10-13,437.18,438.38,431.79,433.41,433.41,4041200
2023-10-16,435.74,439.08,435.53,437.92,437.92,4656900
2023-10-17,434.73,440.05,434.40,438.05,438.05,4864000


In [6]:
# Construct a Kalman filter
kf = KalmanFilter(transition_matrices = [1],    # The value for At. It is a random walk so is set to 1.0
                  observation_matrices = [1],   # The value for Ht.
                  initial_state_mean = 0,       # Any initial value. It will converge to the true state value.
                  initial_state_covariance = 1, # Sigma value for the Qt in Equation (1) the Gaussian distribution
                  observation_covariance=1,     # Sigma value for the Rt in Equation (2) the Gaussian distribution
                  transition_covariance=.01)    # A small turbulence in the random walk parameter 1.0

# Get the Kalman smoothing
state_means, covariances = kf.filter(df['Adj Close'].values)

# Call it KF_mean
df['KF_mean'] = np.array(state_means)
df.tail(20)

,Open,High,Low,Close,Adj Close,Volume,KF_mean
Date,,,,,,,
2023-10-05,426.28,427.23,423.04,426.36,426.36,4030400,434.23
2023-10-06,423.87,433.02,422.46,431.37,431.37,4153500,433.96
2023-10-09,429.45,434.79,428.92,434.21,434.21,2675500,433.98
2023-10-10,434.89,439.15,434.47,436.48,436.48,4430000,434.22
2023-10-11,437.63,438.50,435.10,438.33,438.33,3875600,434.61
2023-10-12,438.91,439.28,433.17,435.70,435.70,4232300,434.72
2023-10-13,437.18,438.38,431.79,433.41,433.41,4041200,434.59
2023-10-16,435.74,439.08,435.53,437.92,437.92,4656900,434.91
2023-10-17,434.73,440.05,434.40,438.05,438.05,4864000,435.21


In [12]:
# first try, filling in empty dates with mean stock price (required for the timeseries datatype
# when frequency is listed as "daily")


fill_val = df['Adj Close'].mean()

#kf_train = kf_train.set_index(['Date'])
kf_train = df[['Adj Close']].copy()
kf_train = ts.from_dataframe(kf_train, value_cols=['Adj Close'],
                             fillna_value=fill_val,freq='D',fill_missing_dates=True)
#kf_train = ts.from_dataframe(kf_sample, value_cols=['GOOGL'])
future_cov = datetime_attribute_timeseries(kf_train, 'day', add_length=4)

model = KalmanForecaster(dim_x=1)
model.fit(kf_train, future_covariates=future_cov)
pred = model.predict(4, future_covariates=future_cov)
preds = pred.values().reshape(-1)
preds

array([419.66363893, 419.79557103, 419.92531823, 420.05288014])